In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#1. INTRODUCTION:

Bella beat, a high-tech manufacturer of health-focused products for women. Bella beat is a successful small company, but they have the potential to become a larger player in the global smart device market. Urška Sršen, co-founder and Chief Creative Officer of Bellabeat, believes that analyzing smart device fitness data could help unlock new growth opportunities for the company

The Bella beat app provides users with health data related to their activity, sleep, stress, menstrual cycle, and mindfulness habits. This data can help users better understand their current habits and make healthy decisions. The Bella beat app connects to their line of smart wellness products

This wellness watch combines the timeless look of a classic timepiece with smart technology to track user activity, sleep, and stress. The Time watch connects to the Bella beat app to provide you with insights into your daily wellness.

Bella beat also offers a subscription-based membership program for users. Membership gives users 24/7 access to fully personalized guidance on nutrition, activity, sleep, health and beauty, and mindfulness based on their lifestyle and goals.



#2. REQUIREMENTS:

##Business Task:

To analyze smart device usage data in order to gain insight into how consumers use non-Bella beat smart devices.

##Questions:

What are some trends in smart device usage?
How could these trends apply to Bella beat Customers?
How could these trends help influence Bella beat marketing strategy?

##Deliverables:

*Clear Summary of the business task.
*Description of all data sources used.
*Documentation of any cleaning or manipulation of data.
*Supporting visualizations and key findings.
*A summary of your analysis.
*Top high-level content recommendations based on your analysis

(I ll be using the existing bellbeat fitness tracker data of sleep tracking, daily activity, weight information)

##1.  Install and load the packages


In [ ]:
install.packages("tidyverse")
install.packages("janitor")
install.packages("knitr")
install.packages("lubridate")
install.packages("dplyr")
install.packages("tidyr")
install.packages("ggplot2")

In [ ]:
library(tidyverse)
library(ggplot2)
library(dplyr)
library(lubridate)
library(tidyr)
library(janitor)
library(knitr)

##2. Importing the data

In [ ]:
daily_activity <- read.csv ("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv")
calories <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyCalories_merged.csv")
sleep <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv")
weight <- read.csv("../input/fitbit/Fitabase Data 4.12.16-5.12.16/weightLogInfo_merged.csv")


##3. summarize the data

In [ ]:
#daily_activity_summary
daily_activity %>%
select(ActivityDate, TotalSteps, SedentaryMinutes, Calories)%>%
summary()

#calories_summary
calories%>%
select(ActivityDay, Calories)%>%
summary()

#sleep_summary
sleep %>%
select(SleepDay, TotalMinutesAsleep, TotalTimeInBed)%>%
summary()

#weight_summary
weight %>%
select(Date, WeightKg, BMI)%>%
summary()

##4. Cleaning the dataset

In [ ]:
daily_activity <- clean_names(daily_activity)
calories <- clean_names(calories)
sleep <- clean_names(sleep)
weight <- clean_names(weight)

##5. checking the colnames and dataset

In [ ]:
str(daily_activity)
head(calories)
str(sleep)
head(weight)

##6. Cleaning the date and time

In [ ]:
daily_activity$activity_date <- mdy(daily_activity$activity_date)
sleep <- separate(sleep, sleep_day, c("date", "time"), " ")
sleep$date <- mdy(sleep$date)
str(daily_activity)
str(sleep)

##7. checking for duplicates

In [ ]:
sum(duplicated(daily_activity))
sum(duplicated(calories))
sum(duplicated(sleep)) #3 duplicates
sum(duplicated(weight))

#removing the duplicates
sleep <- sleep %>% 
distinct()

##8. Plotting the the relationship between steps and calories spent. 
**This visualization show calories burnt increases as the steps increases**

In [ ]:
ggplot(daily_activity) + 
  geom_point(mapping = aes(x = total_steps, y = calories), color = "navy blue") + 
  labs(x = "Steps", y = "calories spent", title = " Steps VS Calories")

##9. Plotting the sleeping trends with the total steps shows there isnt any major relationship between the steps and sleeping trends

In [ ]:
ggplot(daily_activity) + 
  geom_point(mapping = aes(x = total_steps, y = sedentary_minutes), color = "light blue") + 
  labs(x = "Steps", y = "Sleep in minutes", title = " Steps VS Sleep in Minutes")

##10. The above plot doesnt show any relationship between the sleeping trends and total steps. Joining the sleep and daily activity to analyze the other patterns

In [ ]:
sleep_activity_joined <- left_join(sleep,daily_activity, by = c("id" = "id", "date" = "activity_date") )
str(sleep_activity_joined)

In [ ]:
ggplot(sleep_activity_joined) + 
  geom_point(mapping = aes(x = total_minutes_asleep, y = calories), color = "purple") + 
  labs(x = "Minutes Asleep", y = "calories spent", title = " Minutes Asleep VS Calories")

In [ ]:
ggplot(sleep_activity_joined) + 
  geom_point(mapping = aes(x = calories, y = very_active_minutes ), color = "purple") + 
  labs(x = "calories spent", y = " very_active_minutes", title = " Calories vs  very_active_minutes")

This plot shows positive correlation between the calories and steps with very active sedentary minutes.

#Summary

    There is a positive correlation between the calories spent, total steps  & very active minutes. Bellabeat can incorporate this insights in their membership program by introducing activities and goals.